In [ ]:
import torch 
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import WeightedRandomSampler,RandomSampler, DataLoader, random_split
import random
import numpy as np
import os

In [ ]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
dataset = datasets.ImageFolder(root='Rice Leaf Disease Images',transform=transform)

In [ ]:
class_weight=[]
for root, subdir, files in os.walk('Rice Leaf Disease Images'):
    if len(files)>0:
        class_weight.append(1./len(files))
sample_weight = [ class_weight[e] for _,e in dataset.samples]


In [ ]:
sampler = WeightedRandomSampler(sample_weight,num_samples=len(sample_weight),replacement=True)

In [ ]:
total_size = len(dataset)
train_size = int(0.70 * total_size)
val_size =int(0.15 * total_size)
test_size = total_size-train_size-val_size

In [ ]:
train_dataset,val_dataset,test_dataset = random_split(dataset,[train_size,val_size,test_size])

In [ ]:
train_loader = DataLoader(train_dataset,batch_size=6)
test_loader = DataLoader(test_dataset,batch_size=6)
val_loader = DataLoader(val_dataset,batch_size=6)

In [ ]:
model = models.resnet50(pretrained=True)


In [ ]:
num_feature = model.fc.in_features

In [ ]:
num_classes = dataset.classes
model.fc = torch.nn.Linear(num_feature,len(num_classes))

In [ ]:
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [ ]:
device = torch.device("cudo:0" if torch.cuda.is_available() else "cpu")
device

In [15]:
for epoch in range(10):
    model.train()
    running_loss =0.0
    for inputs,labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}')

In [ ]:
torch.save(model,"ResNet50-Rice-Disease-Detection-v1.pt")